# IntelHealth - SFT 数据集生成（Colab）

说明：一次只生成一个数据集。建议先用 `--num_samples 50` 做小样测试，再放大。


In [ ]:
# Clone repo + pull latest
import os
repo_dir = '/content/Intel_Health'
if not os.path.exists(repo_dir):
    !git clone https://github.com/DemonRain7/Intel_Health.git {repo_dir}
else:
    !git -C {repo_dir} pull
%cd {repo_dir}

In [ ]:
# 安装依赖
!pip -q install openai tiktoken jsonschema loguru tqdm

## 1) 设置环境变量
把你的 OpenAI Key 填进下面（只在 Colab 运行时生效）。


In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = (userdata.get("OPENAI_API_KEY") or os.environ.get("OPENAI_API_KEY", "")).strip()
# 如果你有代理或自建 API 网关：
# os.environ["OPENAI_BASE_URL"] = ""


In [ ]:
# Mount Google Drive and set persistent output path
from google.colab import drive
drive.mount('/content/drive')

SFT_OUTPUT_ROOT = '/content/drive/MyDrive/Code_Project/IntelHealth/datasets/agent_sft'
os.makedirs(SFT_OUTPUT_ROOT, exist_ok=True)
print(f'SFT_OUTPUT_ROOT: {SFT_OUTPUT_ROOT}')

## 2) Preprocess / Symptom Normalizer（多阶段生成）

- **seed 模型**：`gpt-5-mini` 生成主诉 + 结构化字段
- **final 模型**：`gpt-4.1` 生成规范输出

In [ ]:
# 生成 preprocess 数据（一次只跑这一条）
!python -u pipeline/sft/build_preprocess_sft.py \
  --num_samples 1500 \
  --output "$SFT_OUTPUT_ROOT/symptom_normalizer/preprocess_sft_data.jsonl" \
  --seed_model gpt-5-mini \
  --final_model gpt-4.1 \
  --output_format conversation \
  --resume \
  --price_in_seed 0.25 --price_out_seed 2.00 \
  --price_in_final 2.00 --price_out_final 8.00

## 3) 其它需要 SFT 的 Agent（逐个生成）

- `symptom_quality_grader`
- `rag_relevance_grader`
- `drug_evidence_grader`

一次只运行其中一个。


In [ ]:
# symptom_quality_grader
!python -u pipeline/sft/generate_sft_data.py \
  --agent symptom_quality_grader \
  --num_samples 1500 \
  --output_dir "$SFT_OUTPUT_ROOT/symptom_quality_grader" \
  --model gpt-5-mini \
  --output_format conversation \
  --resume

In [ ]:
# rag_relevance_grader
!python -u pipeline/sft/generate_sft_data.py \
  --agent rag_relevance_grader \
  --num_samples 1500 \
  --output_dir "$SFT_OUTPUT_ROOT/rag_relevance_grader" \
  --model gpt-5-mini \
  --output_format conversation \
  --resume

In [ ]:
# drug_evidence_grader
!python -u pipeline/sft/generate_sft_data.py \
  --agent drug_evidence_grader \
  --num_samples 1500 \
  --output_dir "$SFT_OUTPUT_ROOT/drug_evidence_grader" \
  --model gpt-5-mini \
  --output_format conversation \
  --resume

## 4) Diagnosis SFT（GPT 从 MCQ 生成诊断场景）

使用 `gpt-5.1` 将 MCQ 医学知识转换为符合 pipeline 格式的诊断 JSON 训练数据。
GPT 对每道 MCQ 先判断是否含患者情境，符合则生成症状 + 诊断 JSON（含 results/recommendations/recomm_short），不符合则跳过。

**Single-pass 模式**：遍历全部 MCQ 一次，够 2000 条提前停止；遍历完仍不足也不会重复。

In [ ]:
# Generate diagnosis SFT data (single-pass, GPT judges each MCQ)
# Stops when 2000 samples reached OR all MCQs scanned — whichever comes first
!python -u pipeline/sft/build_diagnosis_sft.py \
  --input "datasets/agent_sft/train_sft.jsonl" \
  --output "$SFT_OUTPUT_ROOT/diagnosis_generator/diagnosis_sft.jsonl" \
  --synthetic_mode \
  --num_samples 2000 \
  --batch_size 5 \
  --model gpt-5.1 \
  --resume